# < Daily Plan Progress dashboard : 데일리 플랜 진행상황 대시보드 >

## 일정 관리를 자동화하고 DB에 데이터를 기록해놓기 만들었습니다.
    
1. 과목명, 목표 페이지수를 커스텀으로 설정합니다.

2. 한 단위의 공부시간이 끝나면 완료한 페이지를 커밋합니다.

3. 매번 커밋시마다, 모든 과목들의 목표량 대비 내가 진행한양이 한 뷰에 시각화됩니다.(마우스를 갖다대면 구체적 분량이 표시됩니다.)

4. 4번 커밋하면, 긴 시간 단위의 휴식을 취하라는 음악이 나옵니다 (뽀모도로 방식 적용)

5. 커밋시마다, DB에 단위 시간에 진행한 과목명, 페이지량이 저장됩니다. -> 이 데이터를 반성과 추후 계획에 반영합니다.

## 1. 유저가 해야할 일 리스트와 완성해야하는 페이지 또는 시간을 입력합니다 : 직관적으로 각 일이 어느 정도 진행되고 남았는지 보여줍니다.
* 총 4번의 커밋이 이루어졌을 때, 음악을 재생하여 휴식을 하라고 이야기해줍니다.
* ( 개인적으로 저는 30분 단위로 4번 공부를 진행하고 한 번 길게 휴식을 취하는, 뽀모도로 방식을 사용했습니다. )

![title](1.png)

## 2. 유저가 커밋할 때 마다 => 디비에 과목, 당시 총 커밋 횟수, 해당 단위 시간에 완성한 페이지 또는 시간이 기록됩니다.
* 플랜을 짤 때, 저의 능력치를 고려해 짜기 위해 데이터를 기록해두었습니다.

![title](3.png)

## 3. 유저가 프로그램 종료를 선택하면 => 하루를 요약하면 프로그램을 종료합니다.

![title](2.png)

## 1. 필요한 모듈 가져오기

In [6]:
# 1. import modules

# 데이터 처리 라이브러리
import pandas as pd

# 데이터 시각화 라이브러리
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objects as go
init_notebook_mode(connected = True)

# 기타 필요 모듈
import datetime
import pygame # 휴식을 위한 음악 알람 재생 모듈

# 데이터 베이스 모듈
import sqlite3

# 데이터 베이스 파일과 연결
conn = sqlite3.connect("/Users/sucky/Desktop/pageTracker.db", isolation_level=None)
c = conn.cursor()

## 2. 목표를 설정하세요 ( 하루 clean code 10페이지  /  youtube 3시간  etc )

In [7]:
answer = "y"
time_dict = dict()

while answer == 'y':
    
    subject = input("which subject is your goal?")
    startPoint = input("from which n th block/page do you start? 0 recommended.")
    endPoint = input("Till which n th block/page do you want to get dones? relative goal recommended.")
    goalRange = [int(startPoint), int(endPoint)]
    
    time_dict[subject] = goalRange
    
    userChoice = input("more subject to go? [y]/n")
    if userChoice == 'n':
        break

print("here is todays goal : ",time_dict)

# 1. 변수 초기화 - 매일 시작 시점에.
commit_count = 0
    # + pomodoro 횟수 초기화
pomodoro = 0
pomodoro_cycle = 0
totalPomodoro = 0

which subject is your goal?클린 코드
from which n th block/page do you start? 0 recommended.0
Till which n th block/page do you want to get dones? relative goal recommended.12
more subject to go? [y]/nn
here is todays goal :  {'클린 코드': [0, 12]}


## 3. 커밋시에 업데이트 또는 체크할 것들을 "pomodoro function"에 정의 - 뽀모도로 횟수 늘리고, 1단위 채우면 음악재생

In [8]:
# 3. define pomodoro function
def pomodoro_update():
    
    global totalPomodoro
    global pomodoro
    global pomodoro_cycle
    
    # update total pomodoro
    totalPomodoro += 1
    pomodoro += 1
    
    if pomodoro != 4:
        print ("good job !!! sucky {} more pomodoro to go before animal rest.".format(4 - pomodoro))
        
    else : # if pomodoro == 4:
        print ("Take a long maximum 30 minutes rest.")
        pomodoro = 0
        pomodoro_cycle += 1
        print ("pomodoro cycle count updated to {}\
               and pomodoro count reset to {}".format(pomodoro_cycle,pomodoro))
        
        pygame.init()
        pygame.mixer.init()
        pygame.mixer.music.load('./mosaic_lolind.wav')
        pygame.mixer.music.play()
        
        
        #pygame.event.wait()

### 3. 무한 루프로 계속해서 실시간으로 입력을 받고 업데이트하기 - 루프 탈출은 : 유저가 그만하겠다고 할 때

In [ ]:
while True:
    
    # 유저에게 과
    print()
    print([i for i in time_dict.keys()], " in option.")
    subject = input("which subject did you do?")
    
    # 과목명이 기존 리스트 대비 잘못 입력받을 경우 다시 물어보기
    while subject not in [i for i in time_dict.keys()]:
        subject = input('please submit subject name in the right from : ', [i for i in time_dict.keys()])
    
    # 잘 입력 받을 경우

    # 1). get User page turning input
    completed_page = int(input("how many units(pages, blocks) done for the {} ? : ".format(subject)))
                
    # 2). 뽀모도로 카운트들을 업데이트 하기
    if totalPomodoro == 0:
         totalCommit = 1
    else:
         totalCommit = totalPomodoro + 1
            
    # 3). DB에 작성할 현재 날짜-시간 가져오기 
    timeForPageTrackingDb = datetime.datetime.now()
    timeForPageTrackingDb = '20' + timeForPageTrackingDb.strftime("%y-%m-%d %H:%M:%S")
        
    # 4). DB에 작성하기 (테이블 없으면 생성후 작성 )
    c.execute(
        "CREATE TABLE IF NOT EXISTS MyProgress( timeForPageTrackingDb TEXT, subject TEXT, totalCommit INTEGER, completed_page INTEGER )"
    )
    c.execute("INSERT INTO MyProgress VALUES(?,?,?,?)",
        (timeForPageTrackingDb, subject, totalCommit , completed_page))
        
    
    # 커밋받은 것에 따라 진행 상황을 => 데이터 시각화
    original_pages = time_dict[subject]
    updated_pages = [(original_pages[0] + completed_page), original_pages[1]]
    time_dict[subject] = updated_pages
    updated_dataframe = pd.DataFrame.from_dict(time_dict,orient = 'index', columns=['succeed', 'duty'])
    
    # 커밋 횟수 업데이트
    commit_count += 1
    
    # 업데이트된 데이터 프레임
    updated_dataframe['subject'] = updated_dataframe.index
    udf = updated_dataframe
    
    fig = {
        'data': [ 
                 go.Bar(x=udf['subject'], y= udf['duty'], marker_color='crimson', opacity= 0.3),
                 go.Bar(x=udf['subject'], y= udf['succeed'], marker_color='lime', opacity = 0.75)
        ],
        'layout': go.Layout(barmode='overlay')}

    # 플랏을 렌더링하기
    print("\n{}번째 커밋 : ".format(commit_count), datetime.datetime.now().strftime('%H:%M:%S'), end = '')
    iplot(fig, show_link=False)
    
    
    # 뽀모도로 업데이트 함수 호출 - 4단위마다 음악 재생
    pomodoro_update()
    
    # [종료 or 계속] 유저에게 프로그램을 종료할 것인지 물어보기
    answer = input("오늘이 모이고 모여 내일이 됩니다. 계속사히렵니까? [y]/ n")
    while answer not in ['y', 'n']:
        answer = input("Dylan answer again. go on or stop? [y]/ n")
        
    if answer == "n":
        break
        
    else:
        commit_count += 1
        if pomodoro == 0:
            pygame.mixer.music.fadeout(60)
            pygame.quit()
        continue
        
        
# page tracker db close        
conn.close()

# make daily report
print("\n\n\n수고하셨습니다.\n오늘의 총 커밋 횟수 : {}".format(commit_count))
print('20'+datetime.datetime.now().strftime('%y년 %m월 %d일 요약'), end='')
iplot(fig, show_link=False)

              



['클린 코드']  in option.
which subject did you do?클린 코드
how many units(pages, blocks) done for the 클린 코드 ? : 3

1번째 커밋 :  09:56:50

good job !!! sucky 3 more pomodoro to go before animal rest.
